<a href="https://colab.research.google.com/github/DhanieArdiyan/ASE2306-Capstone-Project/blob/main/Capstone_Aircraft_Cabin_Thermal_Comfort_PMV_PPD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
pip install pythermalcomfort


# PMV PPD MODEL

The PMV (Predicted Mean Vote) is the most recognized thermal comfort model, an index and an indicator, that aims to predict the mean value of votes of a group of occupants on a seven-point thermal sensation scale (+3 means that it is too hot, while -3 means its too cold). One can figure out through the PMV whether a person is feeling cold or hot, also explaining why they feel that way. 

The PMV model helps the user to understand the environment parameters to see how it affects the people, and how the different parameters may affect the human. There is a complex set of calculations done to calculate the PMV for a specific environment based on ISO 7730’s standards. These formulas are automatically computed in the pythermalcomfort library which will be imported.

The Predicted Percentage of Dissatisfied, PPD, describes the number of people in percentage, who are uncomfortable at a given PMV index. This would be done once the PMV is calculated to give a quantitative prediction. The contributing factors to cause discomfort includes unwanted cooling or heating of an occupant’s body. Therefore, it is paramount to understand drafts, vertical temperature differences between the ankles and head, and floor temperature, which are the parameters involved in the programming.


In [7]:
# Predicted Mean Vote (PMV) and Predicted Percentage of Dissatisfied (PPD) calculated in accordance to main thermal comfort standards. 
# PMV: Index to predict the mean value of the thermal sensation votes (self-reported perceptions) of a large group of people.
# PPD: Index to establish a quantitative prediction of the percentage of thermally dissatisfied people determined from PMV.

from pythermalcomfort.models import pmv_ppd
from pythermalcomfort.models import v_relative
import warnings 
warnings.filterwarnings('ignore')

#Calculating relative air velocity
vr = v_relative(v=0.1, met=1.0)
# ^ edit this accordingly based on diff metabolism rate of people 

results = pmv_ppd(tdb=24, tr=24, vr=vr, rh=25, met=1.0, clo=0.77, wme=0, standard="ISO")
print(results)

# The relative air velocity can be calculated using the function:
#	(py:meth:'pythermalcomfort.psychrometrics.v_relative')

# ASHRAE Standard Use of the PMV model is limited to air speeds below 0.20m/s (40 fpm).
# When air speeds exceed 0.20 m/s (40 fpm), the comfort zone boundaries are adjusted based on the SET model (Not applicable)

{'pmv': -0.37, 'ppd': 7.8}



**PARAMETERS EXPLAINED**


**tdb** (dry bulb air temp in [°C]): Basically refers to ambient temperature. Ambient temperature is kept between 22-24°C inside a typical aircraft.

**tr**: Mean Radiant Temperature in [°C]. This is a measure of the average temperature of the surfaces that surround a particular point, with which it will exchange thermal radiation. Generally equates the same as tdb.

**vr**: Relative Air Velocity, default in [m/s]. vr is caused by body movement and not the air speed measured by the air velocity sensor (2 diff things). The standard is kept at 0.2m/s.

**rh**: Relative Humidity in [%]. It is kept at 25% max for modern aircrafts (like B787). It would be lower in older planes.

**met**: Metabolic Rate [met]. The metabolic rate for different activities that a human does can be referred to the picture below. For this case, we will apply a metabolic rate of 1.0, corresponding to a person seated.

**clo**: Clothing Insulation [clo]. refers to the different clothing insulation based on typical outfits worn by passengers.

**wme**: External Work [met], default 0 (passengers are not doing strenous work)

**standard**: ISO/ASHRAE (diff equations are used)

![met rate](https://drive.google.com/uc?id=1NuhLjMAxukeT1VLtAJFuH086GMQ6ENJ_)





**ACCEPTABLE RANGES**

In order to comply with the ASHRAE 55 standards, the recommended thermal limit output on the scale has to be between 
**-0.5 to 0.5**.

Humans also have a subjective opinion on what thermal comfort is and what they think is thermally comfortable. This makes it difficult to satisfy everyone, therefore in order to comply with standards, it is acceptable to PPD ranges **below 20%**.

*ASHRAE 55 defines thermal comfort as “that condition of mind that expresses satisfaction within the thermal environment”. 

**EVALUATION OF RESULTS**

We can observe from the results that it returns a PMV value -0.37 and PPD value of 7.8. We can infer from these results that the person is feeling slightly cold, but is still comfortable as it falls under the recommended range.

However, the parameter values can be changed respectively if we want to find out how each individual variable affects the whole model for eg. passenger wearing different set of clothings.

# CLOTHING INSULATION

The following part calculates the different clothing insulation based on typical outfits worn by passengers. The insulation values are referred from the pictures attached.

For the purpose of an example, we will take account of a lady passenger that has the following choice of clothing:

Single breasted coat + Bra + Long sleeve shirt (thin) + Thin trousers + Womens underwear + Ankle socks + Boots


In [4]:
#Calculating the total clothing insulation

from pythermalcomfort.utilities import met_typical_tasks 
#If want to print the indiv task to see the metabolism rate, print seperately
#print(met_typical_tasks['the_activity'])

from pythermalcomfort.utilities import clo_typical_ensembles #clothings
from pythermalcomfort.utilities import clo_individual_garments 

i_cl = clo_individual_garments['Long sleeve shirt (thin)'] + clo_individual_garments['Bra'] + clo_individual_garments['Single-breasted coat (thin)'] + clo_individual_garments['Ankle socks'] + clo_individual_garments["Women's underwear"] + clo_individual_garments['Boots'] 

print(i_cl)

0.77


Through this outfit, we can get a clothing insulation value of 0.77, which holds the value for the variable 'clo' that is computed into the main program.


![clothing of indiv garments](https://drive.google.com/uc?id=1Klkv_ZEWUF2KwonjYttaoIvj-8wGs64N)


# VERTICAL TEMPERATURE GRADIENT SATISFACTION 
This program calculates the percentage of thermally dissatisfied people with a vertical temperature gradient between feet and head.

ASHRAE Standard 55 recommends that the difference should not be greater than 3°C for seated occupants in an environment. 

However due to the decreased relative humidity in the aircraft, the vertical temperature gradient between the feet and the head should not be more than 1.6°C/m. Any temperature difference higher than 1.6°C/m would cause thermal discomfort, with the passengers being cold.

    FROM vertical_tmp_grad_ppd,
    
    def vertical_tmp_grad_ppd(tdb, tr, vr, rh, met, clo, vertical_tmp_grad, units="SI"):
    This equation is only applicable for vr < 0.2 m/s (40 fps).

    Returns: 
    PPD in %, Acceptability in bol.
    1. ASHRAE STANDARD 55 recommends that the difference not be greater than 3°C for seated passengers.
    2. ASHRAE 55 2017 standard defines that the value of air speed at the ankle level is acceptable if PPD_ad is lower or equal than 5%


In [5]:
# Calculates the percentage of thermally dissatisfied people with a 
# vertical temperature gradient between feet and head.

from pythermalcomfort.models import vertical_tmp_grad_ppd
results = vertical_tmp_grad_ppd(24, 24, 0.2, 25, 1.0, 0.77, 1.6)
print(results)

# 1. ASHRAE STANDARD 55 recommends that the difference not be greater 
#    than 3°C for seated passengers.
# 2. ASHRAE 55 2017 standard defines that the value of air speed at the 
#    ankle level is acceptable if PPD_ad is lower or equal than 5%

{'PPD_vg': 1.5, 'Acceptability': True}


The results return a PPD value of 1.5%, which is reasonable for the passenger to be comfortable with the given input parameters.

# ANKLE DRAFT PPD

Excessive air motion or draft causes undesired local cooling of the human body, with draft being identified as the most annoying factor in most environments. 

A low level of air motion is desirable as it removes the warm moist air that builds around the ankle, but it should be strong enough to remove heat and moisture, yet gentle enough to be unnoticed.
Ideally, it should be kept below 0.25m/s in buildings, but that is based on normal relative humidity of 50%, unlike in an aircraft where the relative humidity is at 25%. 

Therefore, ankle velocity must be kept at below 0.1m/s to keep the PPD below 20%.


    FROM ankle_draft,
    
    def ankle_draft(tdb, tr, vr, rh, met, clo, v_ankle, units="SI"):

    Returns:
    PPD in %, Acceptability: bol
    The ASHRAE 55 2017 standard defines that the value of air speed at the ankle level is acceptable if PPD_ad is lower or equal than 20 %

In [6]:
# Calculates the percentage of thermally dissatisfied people with the ankle draft (0.1 m) above floor level.

from pythermalcomfort.models import ankle_draft
results = ankle_draft(25, 25, 0.2, 50, 1.0, 0.77, 0.25, units="SI")
print(results)


# PARAMETERS in respective order
#     tdb: dry bulb air temp in [°C] 
#     	   Note: The air temperature is the average value over two heights: 
#			     0.6 m (24 in.) and 1.1 m (43 in.) for seated occupants
#          1.1 m (43 in.) and 1.7 m (67 in.) for standing occupants.

#     v_ankle : air speed at the 0.1 m (4 in.) above the floor in [m/s] 

# The ASHRAE 55 2017 standard defines that the value of air speed at the ankle level is acceptable if PPD is lower or equal than 20 %.

{'PPD_ad': 15.0, 'Acceptability': True}


The results return a PPD of 15%, which is still fairly decent, but close to being uncomfortable.